In [1]:
import pandas as pd
df=pd.read_excel("/DS Internship - EDA - Data.xlsx")

In [2]:
df.head()

,Store,Year,Month,Super Division,Sales,SGM,Store Open,Store Close,Outlet Type,State,Climate,Total Sq Ft,Store Modification Date,Store Modification
0,1,2015,2,GIRLS,63635.80,35692.09,1991-03-01,No Close date,Outlet,A1,Frigid,4735.0,2011-10-06 00:00:00,relocation
1,1,2015,2,BOYS,20872.93,15691.34,1991-03-01,No Close date,Outlet,A1,Frigid,4735.0,2011-10-06 00:00:00,relocation
2,1,2015,2,ACCESSORIES,38717.03,23866.25,1991-03-01,No Close date,Outlet,A1,Frigid,4735.0,2011-10-06 00:00:00,relocation
3,1,2015,2,KIDS,11455.00,12110.00,1991-03-01,No Close date,Outlet,A1,Frigid,4735.0,2011-10-06 00:00:00,relocation
4,1,2015,3,GIRLS,67507.77,42237.79,1991-03-01,No Close date,Outlet,A1,Frigid,4735.0,2011-10-06 00:00:00,relocation


In [3]:
#1a
total_sales = df.groupby("Year")["Sales"].sum().reset_index()
total_sales


,Year,Sales
0,2015,1.627585e+09
1,2016,1.843939e+09
2,2017,1.651322e+09
3,2018,1.492340e+09
4,2019,1.471963e+09
5,2020,6.510899e+07


In [4]:
#1b
stores_opened1991= df[df["Store Open"].dt.year == 1991]["Store"].nunique()
print("stores opened in 1991 are:",stores_opened1991," stores")



stores opened in 1991 are: 4  stores


In [5]:
#1c
stores_remodeled=df["Store Modification"].notna().sum()
stores_remodeled

181237

In [6]:
#1d
salescorr=df["Sales"].corr(df["Total Sq Ft"])
salescorr


0.13922544019446748

In [7]:
#1e
profitable_division = df.groupby("Super Division")["Sales"].sum().reset_index().sort_values("Sales", ascending=False).head(1)
profitable_division

,Super Division,Sales
2,GIRLS,3.572426e+09


In [8]:
#1f
active_stores=df[df["Store Close"]=="No Close date"]["Store"].nunique()
active_stores

678

In [9]:
#1g
avg_sqft_by_division= df.groupby("Super Division")["Total Sq Ft"].mean().reset_index().sort_values("Total Sq Ft", ascending=False).head(1)
avg_sqft_by_division

,Super Division,Total Sq Ft
3,KIDS,6344.792952


In [10]:
#2a Advanced insights
df["Store Open"] = pd.to_datetime(df["Store Open"], errors='coerce')
df["Store Close"] = pd.to_datetime(df["Store Close"], errors='coerce')
df["Store Modification Date"] = pd.to_datetime(df["Store Modification Date"], errors='coerce')

<ipython-input-10-a5e55c6c99d5>:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["Store Close"] = pd.to_datetime(df["Store Close"], errors='coerce')


In [11]:
state_performance = df.groupby("State").agg(
    total_sales=("Sales", "sum"),
    active_stores=("Store Close",lambda x: x.isna().sum()),
    avg_sq_ft=("Total Sq Ft", "mean")
).reset_index()
state_performance.head(5)

,State,total_sales,active_stores,avg_sq_ft
0,A1,3.738017e+07,710,5573.006236
1,A10,2.694839e+08,5172,6367.403152
2,A2,2.375814e+08,4300,6459.215333
3,A3,3.946254e+08,7208,6001.663180
4,A4,2.738552e+08,5496,6289.636738


In [12]:
state_performance["score"]=(state_performance["total_sales"]/1e9)+state_performance["active_stores"]+(state_performance["avg_sq_ft"]/1000)
top_3=state_performance.sort_values("score",ascending=False).head(3)


In [13]:


top_3

,State,total_sales,active_stores,avg_sq_ft,score
11,B10,866596553.3,16451,5920.746473,16457.787343
18,B8,472226869.2,9704,6521.288556,9710.993515
5,A5,458804278.1,8541,6764.964535,8548.223769


In [14]:
#2b
monthly_sales=df.groupby("Month")["Sales"].mean().reset_index().sort_values("Sales", ascending=False)
best_month=monthly_sales.head(1)
best_month

,Month,Sales
11,12,63392.905721


In [15]:
#2c
outlet_closure_rate=df.groupby("Outlet Type").agg(
    total_stores=("Store", "count"),
    closed_stores=("Store Close", lambda x:x.notna().sum())).reset_index()

In [16]:
outlet_closure_rate["closure_rate"] = (outlet_closure_rate["closed_stores"] / outlet_closure_rate["total_stores"]) * 100
outlet_closure_rate.sort_values("closure_rate",ascending=False)

,Outlet Type,total_stores,closed_stores,closure_rate
1,Outlet,21486,4845,22.549567
2,Strip,73518,13786,18.751870
0,Mall,86233,5122,5.939721


In [17]:
# MODULE 4 REGRESSION
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression


In [18]:
#missing vals
print(df.isnull().sum())

Store                           0
Year                            0
Month                           0
Super Division                  0
Sales                           0
SGM                             0
Store Open                      0
Store Close                157484
Outlet Type                     0
State                           0
Climate                         0
Total Sq Ft                     0
Store Modification Date    105820
Store Modification              0
dtype: int64


In [40]:
df['Store Close'].fillna('Active', inplace=True)
df['Store Modification Date'].fillna('No Modification', inplace=True)


In [20]:
#dropping columns
df.drop(['Store Open', 'Store Close', 'Store Modification Date'], axis=1, inplace=True)


In [21]:
#encoding
categorical_cols = ['Super Division', 'Outlet Type', 'State', 'Climate', 'Store Modification']
df = pd.get_dummies(df, columns=categorical_cols, drop_first=True)


In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 181237 entries, 0 to 181236
Data columns (total 63 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   Store                          181237 non-null  int64  
 1   Year                           181237 non-null  int64  
 2   Month                          181237 non-null  int64  
 3   Sales                          181237 non-null  float64
 4   SGM                            181237 non-null  float64
 5   Total Sq Ft                    181237 non-null  float64
 6   Super Division_BOYS            181237 non-null  bool   
 7   Super Division_GIRLS           181237 non-null  bool   
 8   Super Division_KIDS            181237 non-null  bool   
 9   Outlet Type_Outlet             181237 non-null  bool   
 10  Outlet Type_Strip              181237 non-null  bool   
 11  State_A10                      181237 non-null  bool   
 12  State_A2                      

In [23]:
X = df.select_dtypes(include=['number', 'bool'])  # Include boolean columns as well
# Drop the 'Sales' column from the features
if 'Sales' in X.columns:
    X = X.drop('Sales', axis=1)
y = df['Sales']


In [24]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

print(f'Train set: {X_train.shape}, Test set: {X_test.shape}')


Train set: (144989, 62), Test set: (36248, 62)


In [25]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error, r2_score
lr=LinearRegression()



In [26]:
#Fitting the model
lr.fit(X_train, y_train)
#Making predictions
y_pred= lr.predict(X_test)



In [27]:
mape = mean_absolute_percentage_error(y_test, y_pred)*100
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)
print(f"Model Performance Metrics:")
print(f"MAPE: {mape:.2f}%")
print(f"RMSE: {rmse:.2f}")
print(f"R² Score: {r2:.4f}")


Model Performance Metrics:
MAPE: 6.62%
RMSE: 4254.97
R² Score: 0.9788


In [28]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [29]:
from sklearn.linear_model import Ridge

ridge = Ridge(alpha=1.0)
ridge.fit(X_train, y_train)
print("Ridge Model R²:", ridge.score(X_test, y_test))


Ridge Model R²: 0.9788282810420049


In [30]:
y_pred_ridge = ridge.predict(X_test)
#Evaluate model performance
mape_ridge = mean_absolute_percentage_error(y_test, y_pred_ridge) * 100
r2_ridge = r2_score(y_test, y_pred_ridge)
rmse_ridge= np.sqrt(mean_squared_error(y_test, y_pred_ridge))
print(f"Ridge Model Performance Metrics:")
print(f"MAPE: {mape_ridge:.2f}%")
print(f"RMSE: {rmse_ridge:.2f}")
print(f"R² Score: {r2_ridge:.4f}")


Ridge Model Performance Metrics:
MAPE: 6.62%
RMSE: 4254.97
R² Score: 0.9788


In [31]:
feature_names=X_train.columns
#Extract and display feature importance(coefficients)
ridge_coefficients=pd.DataFrame({'Feature':feature_names,'Coefficient':ridge.coef_})
# Sort by absolute coefficient value
ridge_coefficients['Absolute Coefficient']=np.abs(ridge_coefficients['Coefficient'])
ridge_coefficients = ridge_coefficients.sort_values(by='Absolute Coefficient', ascending=False)
print(ridge_coefficients.head())


                 Feature  Coefficient  Absolute Coefficient
6   Super Division_GIRLS  6558.094677           6558.094677
7    Super Division_KIDS -2155.996867           2155.996867
44              State_D5  -811.014278            811.014278
5    Super Division_BOYS  -785.987263            785.987263
24              State_B5   625.719693            625.719693


In [32]:
#4converting to excel
from openpyxl import Workbook


In [33]:
wb=Workbook()
ws=wb.active
ws.title="Model Summary"


In [34]:
ws.append(["Metric", "Value"])
ws.append(["MAPE (%)", round(mape, 2)])
ws.append(["RMSE", round(rmse, 2)])
ws.append(["R² Score", round(r2, 4)])


In [40]:
ws2 = wb.create_sheet("Key Drivers")
ws2.append(["Feature", "Coefficient", "Absolute Coefficient"])
for row in ridge_coefficients.itertuples(index=False):
    ws2.append(row)

wb.save("ridge_model_summary.xlsx")
